# Severstal: Steel Defect Detection

https://www.kaggle.com/c/severstal-steel-defect-detection

In [96]:
import os
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader, sampler
from torchvision import transforms as T
from torchvision import models

from PIL import Image

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

from datetime import datetime

from severstal_utils import *
from severstal_models import *

In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
TRAIN_PATH = 'train_images'
TEST_PATH = 'test_images'
MODEL_PATH = 'models/'
SUBMS_PATH = 'submissions/'


if torch.cuda.is_available():
    device = torch.device('cuda')
    dtype = torch.cuda.FloatTensor
else:
    device = torch.device('cpu')
    dtype = torch.FloatTensor

In [93]:
img_mean = (0.3438812517320017,)
img_std = (0.13965334396720055,)

transform_train = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=img_mean, std=img_std)
])

transform_val = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=img_mean, std=img_std)
])

In [95]:
# Splitting to training and validation parts
imglist = os.listdir(TRAIN_PATH)
imglist_train, imglist_val = train_test_split(imglist, test_size=0.2, random_state=0)

# Datasets
ds_train = SteelFramesDataset(TRAIN_PATH, imglist_train, masks_csv='train.csv', transform=transform_train)
ds_val = SteelFramesDataset(TRAIN_PATH, imglist_val, masks_csv='train.csv', transform=transform_val)

In [99]:
params_loader = {
    'batch_size': NUM_FRAMES,
    'num_workers': 4,
    'pin_memory': True
}

# Data loaders
loader_train = DataLoader(
    ds_train,
    shuffle=True,
    **params_loader
)
loader_val = DataLoader(
    ds_val,
    shuffle=False,
    **params_loader)

In [105]:
# Model defenition
params_unet = {
    'depth': 5,
    'n_filters': 16,
    'bn': True,
    'activation': 'relu'
}

unet = UnetD(**params_unet)
unet = unet.to(device=device);

detector = Detector(unet, device)

In [ ]:
%%time

params_train = {
    'epochs': 2,
    'print_every': 1000,
    'lr': 0.0001,
}

loss_fun = torch.nn.CrossEntropyLoss()

detector.fit(loader_train, loader_val, loss_fun, **params_train)

After 1001 frames processed
Trainig loss = 1.6056696201537872
Validation loss: 1.4093183311281834
After 2002 frames processed
Trainig loss = 1.3344863161340461
Validation loss: 1.2702207498922173
After 3003 frames processed
Trainig loss = 1.186470256818758
Validation loss: 1.1190422101919861
After 4004 frames processed
Trainig loss = 1.057180233351834
Validation loss: 0.9897314707438151
After 5005 frames processed
Trainig loss = 0.9361093469433017
Validation loss: 0.8668374378432712
After 6006 frames processed
Trainig loss = 0.8175276409495961
Validation loss: 0.77199826270413
After 7000 frames processed
Trainig loss = 0.7291654214053087
Validation loss: 0.6761268690501117
After 8001 frames processed
Trainig loss = 0.6406407689714765
Validation loss: 0.5908665621242736
After 9002 frames processed
Trainig loss = 0.5733173264490141
Validation loss: 0.5295489358826109
After 10003 frames processed
Trainig loss = 0.4919619568578013
Validation loss: 0.4591265853521079
After 11004 frames proc

In [ ]:
plt.plot(
    detector.n_frames, detector.loss_history,
    detector.n_frames, detector.val_loss_history,
);